In [ ]:
 from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from bs4 import BeautifulSoup
import numpy as np
import base64
import re
import pandas as pd
from tqdm import tqdm

In [ ]:
# чтобы это запустилось, нужно сделать что написано здесь https://developers.google.com/gmail/api/quickstart/python
# часть кода взята отсюда https://www.geeksforgeeks.org/how-to-read-emails-from-gmail-using-gmail-api-in-python/

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

creds = None
if os.path.exists('token_slava.json'):
    creds = Credentials.from_authorized_user_file('token_slava.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials_slava.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token_slava.json', 'w') as token:
        token.write(creds.to_json())

service = build('gmail', 'v1', credentials=creds)
  
result = service.users().messages().list(userId='me', maxResults=100).execute()

messages = result.get('messages')

In [ ]:
sources = ['Открытие', 'Movchan', 'АТОN RESEARCH', 'Bloomberg', 'ROSBANK Research']

for n in tqdm(range(28)):
    if n != 0:
        page_token = result['nextPageToken']
        result = service.users().messages().list(userId='me', pageToken=page_token).execute()
        messages = result.get('messages')
    for msg in messages:
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
        payload = txt['payload']
        headers = payload['headers']

        for d in headers:
            if d['name'] == 'Subject':
                subject = d['value']
            if d['name'] == 'From':
                sender = d['value']
            if d['name'] == 'Date':
                date = d['value']
        if any([source in sender for source in sources]):
            try:
                parts = payload.get('parts')[0]
            except:
                pass
            try:
                data = parts['body']['data']
            except:
                data = parts.get('parts')[0]['body']['data']

            data = data.replace("-","+").replace("_","/")
            decoded_data = base64.b64decode(data)
            soup = BeautifulSoup(decoded_data , "lxml")
            body = soup.body()

            if 'Открытие' in subject:
                lines = re.sub('\s+', ' ', soup.get_text(strip=True))
                lines = lines[lines.find('Главное'):lines.find('Читать')]
            elif 'ROSBANK Research' in sender:
                lines = re.sub('\s+', ' ', soup.get_text(strip=True))
                lines = lines[:lines.find('Напишите нам')]
            elif 'Bloomberg' in sender:
                lines = re.sub('\s+', ' ', soup.get_text(strip=True))
                lines = lines[:lines.find('You received this message')]
            elif 'АТОN RESEARCH' in sender:
                lines = re.sub('\s+', ' ', soup.get_text(strip=True))
                lines = lines[lines.find('Рынок акций'):lines.find('Все права защищены')]
            elif 'Movchan' in sender:
                lines = re.sub('\s+', ' ', soup.get_text(strip=True))
                lines = lines[:lines.find('Подписаться на рассылку')]
            words = [date, lines]
            with open('записки.txt', 'a') as f:
                for line in words:
                    try:
                        f.write(line)
                        f.write('\n')
                    except:
                        pass
                f.write('\n')